<header style="color:green;font-size:30px; text-align:center;">      
<b>
    FOREST GUARDS
</b>
</header>   

### Import and authentification

In [ ]:
import ee 
import tensorflow as tf
from IPython.display import Image

%load_ext autoreload
%autoreload 2

In [ ]:
# TO BE DONE IN GOOGLE COLAB ONLY


!git clone https://github.com/TheChwal/forest_guard.git

%cd forest_guard/

!pip install .

In [ ]:
#TO BE DONE IN GOOGLE COLAB ONLY

from google.colab import auth

auth.authenticate_user()

In [ ]:
# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize()


### Global variables

In [ ]:

BUCKET = 'forest_guard_bucket'  # ⚠️ replace with your BUCKET NAME
FOLDER = 'data_trainval' # ⚠️ replace with your folder
TRAINING_BASE = 'training_patches'
EVAL_BASE = 'eval_patches'

OPTICAL_BANDS = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
THERMAL_BANDS = ['B10', 'B11']
BANDS = OPTICAL_BANDS + THERMAL_BANDS
RESPONSE = 'fnf'
FEATURES = BANDS + [RESPONSE]

# Specify the size and shape of patches expected by the model.
KERNEL_SIZE = 256
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]

COLUMNS = [
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURES
]
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))

  

### Exporting training and val set 

#### Processing and Visualising the data on folium

##### X

In [ ]:
from forest_guard.datas import get_image_landsat

# The image input data is a cloud-masked median composite.
image = get_image_landsat('2015-01-01', '2017-12-31')


# Use folium to visualize the imagery.
mapid = image.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3})
map = folium.Map(location=[48.9, 2.5])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='median composite',
    opacity = 0.5
  ).add_to(map)

mapid = image.getMapId({'bands': ['B11'], 'min': 0, 'max': 0.5})
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='thermal',
    opacity = 0.5
  ).add_to(map)
map.add_child(folium.LayerControl())
map

##### Target

In [ ]:
from forest_guard.datas import get_jaxa
jaxa = get_jaxa('2017-01-01', '2017-12-31')

palette = ['006400',
           'FEFF99',
            '0000FF'
          ]
mapid = jaxa.getMapId({'bands': ['fnf'],
                       'min': 1, 
                       'max': 3, 
                      'palette':palette
                      })
map = folium.Map(location=[48.9, 2.5])


folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='jaxa fnf',
    color=palette,
    opacity = 1
  ).add_to(map)
map.add_child(folium.LayerControl())
map

#### Stacking X and y

In [ ]:
from forest_guard.datas import get_stacked_arrays

arrays = get_stacked_arrays(image, jaxa, BANDS, RESPONSE, KERNEL_SIZE)

#### Areas of interests for training and test

***For now, X and y are virtual satellite data for the full space***  
It is need to choose the regional areas where we are going to train and test

In [ ]:
# create our own areas of interests


# training features collection
rect_train = []
#massif central
rect_train.append(ee.Geometry.Rectangle([3.327248985602229, 46.600827791084875, 4.689553673102229,47.70649093701327]))
#west alpes
rect_train.append(ee.Geometry.Rectangle([5.7423828124999865,45.351141225773794, 8.291210937499986,46.633369678740046]))
#sweden
rect_train.append(ee.Geometry.Rectangle([12.158398437499986,57.85568314226695, 14.883007812499986,59.23144539808598]))
#sydney area
rect_train.append(ee.Geometry.Rectangle([150.04780273437498,-34.28117194241202, 151.47602539062498,-33.11122308232406]))

trainingPolys = ee.FeatureCollection(rect_train)

# eval features collections
rect_eval=[]
#west France
rect_eval.append(ee.Geometry.Rectangle([-0.7016469211726672,46.88962161312492, 0.9353159694523328,47.64998631126759]))
#east alpes
rect_eval.append(ee.Geometry.Rectangle([10.319052480734172,45.60830293723585, 13.527060293234172,47.511663948188954]))
#russia
rect_eval.append(ee.Geometry.Rectangle([29.61104466823417,57.26094864586795, 32.68721654323417,58.93274292154694]))
#australia
rect_eval.append(ee.Geometry.Rectangle([149.36943235990577,-37.37123372588969, 150.22636595365577,-36.73996965953723]))

evalPolys= ee.FeatureCollection(rect_eval)

In [ ]:
###################################################
###   DEFINE OUR COLLECTIONS OF AREAS OF INTEREST
################################################

polyImage = ee.Image(0).byte().paint(trainingPolys, 1).paint(evalPolys, 2)
polyImage = polyImage.updateMask(polyImage)

mapid = polyImage.getMapId({'min': 1, 'max': 2, 'palette': ['red', 'blue']})
map = folium.Map(location=[47., 0.], zoom_start=5)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='training polygons',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

#### Export data in TFRecords format

In [ ]:
from forest_guard.datas import export_to_cloud_sampling_train_eval


export_to_cloud_sampling_train_eval(trainingPolys, evalPolys, arrays, 100, 1000, FOLDER)

### Parse TFRecords and get back tensors

#### Training set

In [ ]:
from forest_guard.parse import get_training_dataset

training = get_training_dataset(FOLDER)

print(iter(training.take(1)).next()) # to visualise one element

#### Visualise training set

In [ ]:
images , labels = iter(training.take(1)).next()

fig = plt.figure(figsize=(15,7))

for i in range(5):
    plt.subplot(1, 5, i+1 )
    plt.imshow(np.flip(images[i, :, : , 1:4].numpy(), axis=2)*5)
    

In [ ]:
fig = plt.figure(figsize=(15,7))

for i in range(5):
    plt.subplot(1,5,i+1)
    plt.imshow(labels[i, :, : , :])


#### Training Data augmentation

***If need horizontal and vertical flips are possible***  
But this is is heavy in memory

In [ ]:
from forest_guard.parse import get_training_dataset_gen

training_gen = get_training_dataset_gen(FOLDER)


#### Visualise augmented training set

In [ ]:
images , labels = next(training_gen)
images.shape, labels.shape

In [ ]:
fig, axes = plt.subplots(2,4, figsize=(15,7))
   
for i in range(4):
    axes[0,i].imshow(np.flip(images[i, :, : , 1:4], axis=2)*5)
    axes[1,i].imshow(labels[i, :, : , :])
    

#### Evaluation set

In [ ]:
from forest_guard.parse import get_eval_dataset
evaluation = get_eval_dataset(FOLDER)

In [ ]:
print(iter(evaluation.take(1)).next())

#### Evaluation augmentation

In [ ]:

from forest_guard.parse import get_eval_dataset_gen
evaluation_gen = get_eval_dataset_gen(FOLDER)

In [ ]:
images , labels = next(evaluation_gen)
images.shape, labels.shape

### Model training and performance monitoring

#### Training

***Below is a way to train from the notebook (Colab)***  
Be careful that GPU of Colab are not at the same place of where your data are stored, which implies heavy costs  
***Training on AI platform is possible*** to avoid this issue

In [ ]:
from forest_guard.trainer import Trainer

print('\n', 'instantiate trainer')
trainer = Trainer('model_name')

Dice, tversky and lovasz_softmax are 3 losses implemented
***lovasz_softmax assumes that output of the model is in logit space***

In [ ]:
from forest_guard.keras.losses import iou
from forest_guard.losses import dice_loss
from forest_guard.losses import tversky_loss
from forest_guard.losses import lovasz_softmax

tversky = tversky_loss(0.75)
dice = dice_loss
lovasz = lovasz_softmax

It is possible to download a model already existing or create a Unet from scratch

In [ ]:
download = False
if download:
    print('\n', 'download model', '\n')
    trainer.download_model_from_gcp()
else:
    print('\n', 'build model', '\n')
    trainer.init_model() #put sigmoid at false for lovasz softmax

In [ ]:
trainer.model.summary()

***Train the model***

In [ ]:

print('\n', 'run trainer', '\n')
history = trainer.run(training,
                      evaluation,
                      100,
                      metrics = [iou, "mae", "accuracy"], 
                      optimizer='adam',
                      loss=tversky,
                      train_size = 3200,
                      eval_size=1600,
                     patience=10)


In [ ]:
# write metrics in ml flow
trainer.metrics_to_mlflow(history)


#save history on the cloud
trainer.save_history(history)


# save model in your bucket

print('\n', 'save model', '\n')
trainer.save_model()

#### Visualise training history

In [ ]:
from forest_guard.history import get_history_colab
from forest_guard.history import plot_history_accuracy
from forest_guard.history import plot_history_iou



In [ ]:
from forest_guard.history import get_history_colab
hist = get_history_colab('model_name')
plot_history_accuracy(hist)
plot_history_iou(hist)

### _EEfying_ the model and push it on AI platform

In [ ]:
MODEL_DIR = 'gs://forest_guard_bucket/models/forest_guard/model_name'
EEIFIED_DIR = 'gs://forest_guard_bucket/models/forest_guard/eefied'
REGION = 'us_central1'

In [ ]:
from tensorflow.python.tools import saved_model_utils

meta_graph_def = saved_model_utils.get_meta_graph_def(MODEL_DIR, 'serve')
inputs = meta_graph_def.signature_def['serving_default'].inputs
outputs = meta_graph_def.signature_def['serving_default'].outputs

# Just get the first thing(s) from the serving signature def.  i.e. this
# model only has a single input and a single output.
input_name = None
for k,v in inputs.items():
  input_name = v.name
  break

output_name = None
for k,v in outputs.items():
  output_name = v.name
  break

# Make a dictionary that maps Earth Engine outputs and inputs to 
# AI Platform inputs and outputs, respectively.
import json
input_dict = "'" + json.dumps({input_name: "array"}) + "'"
output_dict = "'" + json.dumps({output_name: "fnf"}) + "'"

In [ ]:
PROJECT = 'my-project'

In [ ]:
# You need to set the project before using the model prepare command.

!earthengine set_project {PROJECT}
!earthengine model prepare --source_dir {MODEL_DIR} --dest_dir {EEIFIED_DIR} --input {input_dict} --output {output_dict}

Model is _eefied_, needs now to be pushed on AI platform for predictions

In [ ]:
MODEL_NAME = 'model_name'

In [ ]:
VERSION_NAME = 'v' + str(int(time.time()))
print('Creating version: ' + VERSION_NAME)

!gcloud ai-platform models create {MODEL_NAME} \
  --project {PROJECT} \
  --region {REGION}

!gcloud ai-platform versions create {VERSION_NAME} \
  --project {PROJECT} \
  --model {MODEL_NAME} \
  --region {REGION} \
  --origin {EEIFIED_DIR} \
  --framework "TENSORFLOW" \
  --runtime-version 2.3 \
  --python-version 3.7 \
  #--machine-type mls1-c4-m2
  #--min-node=1 etc \
  # gpu ...

### Visualise prediction in Folium 

In [ ]:
from forest_guard.predictFoliumEE import add_ee_layer
from forest_guard.predictFoliumEE import maskL8sr

folium.Map.add_ee_layer = add_ee_layer
from forest_guard.predictFoliumEE import map_pred

In [ ]:
map = map_pred(MODEL_NAME_SMALL, VERSION_SMALL, REGION, 2017, 2020)

display(map)